### Import Things

In [1]:
import numpy as np
import pandas as pd

### Get Data

In [3]:
skaters = pd.read_csv("skaters.csv")
goalies = pd.read_csv("goalies.csv")
pbp = pd.read_csv("pbp.csv")
pbp.head()

/var/folders/5k/vm4wbtfd3hz2hq37wm4d1_8h0000gp/T/ipykernel_45444/517209628.py:3: DtypeWarning: Columns (45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp = pd.read_csv("pbp.csv")


,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,periodDescriptor.number,periodDescriptor.periodType,...,details.discreteClip,details.servedByPlayerId,gameId,periodDescriptor.otPeriods,pptReplayUrl,details.highlightClipSharingUrl,details.highlightClipSharingUrlFr,details.highlightClip,details.highlightClipFr,details.discreteClipFr
0,51.0,00:00,20:00,1551.0,right,520.0,period-start,8.0,1.0,REG,...,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.0,00:00,20:00,1551.0,right,502.0,faceoff,9.0,1.0,REG,...,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101.0,00:15,19:45,1551.0,right,503.0,hit,10.0,1.0,REG,...,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54.0,00:17,19:43,1551.0,right,506.0,shot-on-goal,11.0,1.0,REG,...,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55.0,00:27,19:33,1551.0,right,507.0,missed-shot,12.0,1.0,REG,...,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Split Play-By-Play Into Train (2021-22, 2022-23, 2023-24) and Test (2024-25), Additionally Filter for Regular Season Only

In [22]:
# Set "gameId" to a string
pbp["gameId"] = pbp["gameId"].apply(str)

# Split into training and testing events
train_pbp = pbp.loc[pbp.gameId.str.startswith(("2021", "2022", "2023"), na=False)]
test_pbp = pbp.loc[pbp.gameId.str.startswith("2024", na=False)]
print("Training events: ", len(train_pbp))
print("Testing events: ", len(test_pbp))

Training events:  1404747
Testing events:  479352


In [36]:
# Keep only regular season games (remove pre-season and playoffs)
train_pbp = train_pbp.loc[train_pbp["gameId"].str[5] == "2"]
test_pbp = test_pbp.loc[test_pbp["gameId"].str[5] == "2"]
print("Training events: ", len(train_pbp))
print("Testing events: ", len(test_pbp))

Training events:  1227064
Testing events:  420073


### Functions to Return Subsets of Events

In [41]:
# Keep only shots and goals function
def keep_SOG(df):
    df = df[(df["typeDescKey"] == "shot-on-goal") | (df["typeDescKey"] == "goal")]

    return(df)

### Krzywicki (2005)

In [ ]:
# Variables
# - Distance (<10ft, 10ft-12ft, 13ft-14ft, 15ft-16ft, 17ft-22ft, 23ft-31ft, 32ft-36ft, 37ft-38ft
#             39ft-44ft, 45ft-57ft, >=58ft)
# - Shot Type (Wrap, Slap, Wrist, Snap, Backhand, Tip)
# - Rebound (Yes, No)
# - Situation (Even Strength, Shorthanded, Powerplay)

In [43]:
train_pbp_krzy05 = keep_SOG(train_pbp)
test_pbp_krzy05 = keep_SOG(test_pbp)
print("Training shots: ", len(train_pbp_krzy05))
print("Testing shots: ", len(test_pbp_krzy05))

Training shots:  246134
Testing shots:  74641
